In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json
from io import StringIO
import io
import shapely

### Step1 clean

#### 1. weather data - Local Climatological Data (LCD)

In [2]:
df = pd.read_csv('weather130101-171130.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,11,14,15,20,23,25,40,44,45,67,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print ("Shape of data\n{}".format(df.shape))

Shape of data
(55996, 90)


In [4]:
df.drop(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 
              'LONGITUDE', 'REPORTTPYE', 'HOURLYSKYCONDITIONS', 
              'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 
              'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 
              'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 
              'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 
              'HOURLYRelativeHumidity', 'HOURLYWindDirection',
              'HOURLYPrecip', 'MonthlyMaximumTemp',
       'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH',
       'MonthlyDewpointTemp', 'MonthlyWetBulbTemp',
       'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays',
       'MonthlyStationPressure', 'MonthlySeaLevelPressure',
       'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall',
       'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp',
       'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip',
       'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip',
       'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall',
       'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth',
       'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp',
       'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp',
       'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip',
       'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow',
       'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate',
       'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue',
       'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime',
       'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays',
       'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD',
       'MonthlyTotalSeasonToDateHeatingDD',
       'MonthlyTotalSeasonToDateCoolingDD','DAILYAverageStationPressure',
       'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed',
       'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed',
       'DAILYSustainedWindDirection','DAILYMaximumDryBulbTemp',
       'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
       'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity',
       'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp',
       'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays'], axis=1, inplace=True)

In [5]:
df.columns

Index(['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'HOURLYPressureTendency',
       'HOURLYPressureChange', 'HOURLYSeaLevelPressure',
       'HOURLYAltimeterSetting', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth'],
      dtype='object')

In [6]:
df.count()

DATE                      55996
HOURLYWindSpeed           50749
HOURLYWindGustSpeed        9362
HOURLYStationPressure     53604
HOURLYPressureTendency    14083
HOURLYPressureChange       2423
HOURLYSeaLevelPressure    42318
HOURLYAltimeterSetting    53660
DAILYSunrise              55996
DAILYSunset               55996
DAILYWeather                776
DAILYPrecip                2388
DAILYSnowfall              1794
DAILYSnowDepth             1794
dtype: int64

In [7]:
df.head(1)

,DATE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYAltimeterSetting,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth
0,2013-01-01 00:51,10.0,22.0,29.72,8.0,NaN,29.87,29.89,720,1639,NaN,NaN,NaN,NaN


In [8]:
df1 = df[['DATE', 'DAILYSunrise', 'DAILYSunset']]
df1.head(1)

,DATE,DAILYSunrise,DAILYSunset
0,2013-01-01 00:51,720,1639


In [9]:
df1['DATE'] = pd.to_datetime(df1['DATE'], format='%Y/%m/%d')
df1['DATE'] = df1['DATE'].dt.date
df1['DATE'] = pd.to_datetime(df1['DATE'], format='%Y/%m/%d')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [10]:
df1.dtypes

DATE            datetime64[ns]
DAILYSunrise             int64
DAILYSunset              int64
dtype: object

In [11]:
df1.drop_duplicates(subset='DATE', inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1.reset_index(drop=True, inplace=True)
df1.head()

,DATE,DAILYSunrise,DAILYSunset
0,2013-01-01,720,1639
1,2013-01-02,720,1640
2,2013-01-03,720,1641
3,2013-01-04,720,1642
4,2013-01-05,720,1643


In [28]:
df1.shape

(1795, 3)

In [29]:
df1.dtypes

DATE            datetime64[ns]
DAILYSunrise             int64
DAILYSunset              int64
dtype: object

In [30]:
df1.DATE.unique()

array(['2013-01-01T00:00:00.000000000', '2013-01-02T00:00:00.000000000',
       '2013-01-03T00:00:00.000000000', ...,
       '2017-11-28T00:00:00.000000000', '2017-11-29T00:00:00.000000000',
       '2017-11-30T00:00:00.000000000'], dtype='datetime64[ns]')

#### 2. weather data -- daily summary

In [13]:
df2 = pd.read_csv('weather_daily_summary_2013_now.csv')

In [14]:
print ("Shape of data\n{}".format(df2.shape))

Shape of data
(1794, 27)


In [15]:
df2.count()

STATION      1794
NAME         1794
LATITUDE     1794
LONGITUDE    1794
ELEVATION    1794
DATE         1794
AWND         1781
PGTM            1
PRCP         1794
SNOW         1793
SNWD         1793
TAVG            0
TMAX         1794
TMIN         1794
WSF2         1784
WSF5         1768
WT01          508
WT02           32
WT04            5
WT06            7
WT08          347
WT13           27
WT14            1
WT16           23
WT18           21
WT19            9
WT22            2
dtype: int64

In [16]:
df2.head(1)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PGTM,PRCP,SNOW,...,WT02,WT04,WT06,WT08,WT13,WT14,WT16,WT18,WT19,WT22
0,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,2013-01-01,6.93,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df2.dtypes

STATION       object
NAME          object
LATITUDE     float64
LONGITUDE    float64
ELEVATION    float64
DATE          object
AWND         float64
PGTM         float64
PRCP         float64
SNOW         float64
SNWD         float64
TAVG         float64
TMAX           int64
TMIN           int64
WSF2         float64
WSF5         float64
WT01         float64
WT02         float64
WT04         float64
WT06         float64
WT08         float64
WT13         float64
WT14         float64
WT16         float64
WT18         float64
WT19         float64
WT22         float64
dtype: object

In [18]:
df2.drop(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION'], axis=1, inplace=True)
df2.columns = ['DATE', 'wind_speed_average',
       'peak_gust_time', 'precipitation', 'snowfall', 'snow_depth', 'temperature_average', 'temperature_max', 'temperature_min', 'fastest_2minute_windspeed', 'fastest_5minute_windspeed',
       'fog', 'heavy_fog', 'ice_pellets', 'glaze_rime', 'smoke_haze', 'mist', 'drizzle', 'rain', 'WT18',
       'precipitation_unknownsource', 'ice_fog']

In [19]:
df2.drop(['peak_gust_time', 'temperature_average', 'fastest_5minute_windspeed', 'WT18', 'precipitation_unknownsource'], axis=1, inplace=True)

In [20]:
df2.count()

DATE                         1794
wind_speed_average           1781
precipitation                1794
snowfall                     1793
snow_depth                   1793
temperature_max              1794
temperature_min              1794
fastest_2minute_windspeed    1784
fog                           508
heavy_fog                      32
ice_pellets                     5
glaze_rime                      7
smoke_haze                    347
mist                           27
drizzle                         1
rain                           23
ice_fog                         2
dtype: int64

In [21]:
df2.dtypes

DATE                          object
wind_speed_average           float64
precipitation                float64
snowfall                     float64
snow_depth                   float64
temperature_max                int64
temperature_min                int64
fastest_2minute_windspeed    float64
fog                          float64
heavy_fog                    float64
ice_pellets                  float64
glaze_rime                   float64
smoke_haze                   float64
mist                         float64
drizzle                      float64
rain                         float64
ice_fog                      float64
dtype: object

#### 3. Storm data

In [22]:
dfbad = pd.read_csv('storm_data_search_results.csv')
dfbad.head(1)

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_RANGE,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER
0,421883,NEW YORK (MANHATTAN) (ZONE),,01/31/2013,0,High Wind,50.00,,0,0,...,,,,,,,,,,1


In [23]:
dfbad['EVENT_TYPE'].unique()

array(['High Wind', 'Winter Storm', 'Thunderstorm Wind', 'Flash Flood',
       'Flood', 'Excessive Heat', 'Strong Wind', 'Dense Fog',
       'Winter Weather', 'Heavy Snow', 'Blizzard'], dtype=object)

In [24]:
dfbad.shape

(36, 35)

In [25]:
dfbad.columns

Index(['EVENT_ID', 'CZ_NAME_STR', 'BEGIN_LOCATION', 'BEGIN_DATE', 'BEGIN_TIME',
       'EVENT_TYPE', 'MAGNITUDE', 'TOR_F_SCALE', 'DEATHS_DIRECT',
       'INJURIES_DIRECT', 'DAMAGE_PROPERTY_NUM', 'DAMAGE_CROPS_NUM',
       'STATE_ABBR', 'CZ_TIMEZONE', 'MAGNITUDE_TYPE', 'EPISODE_ID', 'CZ_TYPE',
       'CZ_FIPS', 'WFO', 'INJURIES_INDIRECT', 'DEATHS_INDIRECT', 'SOURCE',
       'FLOOD_CAUSE', 'TOR_LENGTH', 'TOR_WIDTH', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION',
       'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON', 'ABSOLUTE_ROWNUMBER'],
      dtype='object')

In [26]:
df3 = dfbad[['BEGIN_DATE', 'EVENT_TYPE']]
df3.head(1)

,BEGIN_DATE,EVENT_TYPE
0,01/31/2013,High Wind


In [27]:
df3.dtypes

BEGIN_DATE    object
EVENT_TYPE    object
dtype: object

### Step2 Merge